In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor, Pool

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
df.head()

Let's add CPM

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)), x['measurable_impressions'])*1000 , axis=1)

In [ ]:
df.isnull().sum()

Remove the outliers 

In [ ]:
# cond = (df['CPM'] < df['CPM'].quantile(.95)) & (df['CPM'] >= 0)
cond = (df['CPM'] >= 0)
df = df[cond]

Remove unnessecary columns

In [ ]:
cols = ['total_revenue', 'order_id' , 'line_item_type_id']
df = df.drop(cols, axis=1)

In [ ]:
# split train-test-val
from sklearn.model_selection import train_test_split

df['timestamp'] = df['date'].apply(pd.Timestamp).astype(int)

df_train = df[df.date < '2019-06-22']
df_test = df[df.date >= '2019-06-22']

cond = (df_test['CPM'] < df_test['CPM'].quantile(.95)) & (df_test['CPM'] >= 0)
df_test = df_test[cond]

cond = (df_train['CPM'] < df_train['CPM'].quantile(.95)) & (df_train['CPM'] >= 0)
df_train = df_train[cond]

df_train.drop('date', axis=1, inplace=True)
df_test.drop('date', axis=1, inplace=True)

X = df_train.drop(['CPM'], axis=1)
y = df_train.CPM

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)

Check for Dtypes

In [ ]:
X_train

In [ ]:
X.info()

In [ ]:
from catboost import CatBoostRegressor, Pool
cat_columns = [0, 1, 2, 3, 4, 5, 6, 7, 8]
train_pool = Pool(X_train, y_train, cat_features=cat_columns)

In [ ]:
model = CatBoostRegressor(
    loss_function = 'RMSE',
    random_seed=42,
    logging_level='Silent',
)

In [ ]:
model.fit(
    train_pool,
    eval_set=(X_val, y_val),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True,
    use_best_model=True,
)

Get predictions

In [ ]:
X_test = df_test.drop(['CPM'], axis=1)
y_test = df_test.CPM

In [ ]:
y_pred = model.predict(X_test)

Check MSE

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)